In [1]:
pip install opencv-python


In [2]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import tensorflow as tf

# ================================
# 1. Load Images and Masks
# ================================
def load_images_and_masks(image_folder, mask_folder, img_size):
    images, masks = [], []

    for filename in os.listdir(image_folder):
        img_path = os.path.join(image_folder, filename)
        mask_path = os.path.join(mask_folder, filename)

        if os.path.exists(img_path) and os.path.exists(mask_path):
            image = cv2.imread(img_path)
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

            if image is not None and mask is not None:
                image = cv2.resize(image, (img_size, img_size)) / 255.0
                mask = cv2.resize(mask, (img_size, img_size)) / 255.0
                mask = np.expand_dims(mask, axis=-1)

                images.append(image)
                masks.append(mask)
            else:
                print(f"Failed to read {filename}")

    print(f"Loaded {len(images)} images and {len(masks)} masks")
    return np.array(images), np.array(masks)

# Paths to dataset
image_folder = r'C:\CINNAMON\dataset\image_folder'
mask_folder = r'C:\CINNAMON\dataset\masks_folder'
img_size = 256

# Load data
X, y = load_images_and_masks(image_folder, mask_folder, img_size)

# ================================
# 2. Data Augmentation
# ================================
data_gen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Split into training and validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ================================
# 3. Dice Loss Function
# ================================
def dice_loss(y_true, y_pred):
    smooth = 1.0
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    return 1 - (2.0 * intersection + smooth) / (union + smooth)

# ================================
# 4. Define U-Net Model
# ================================
def unet_model(input_shape, num_classes):
    inputs = Input(input_shape)

    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(0.1)(p1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(0.1)(p2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(0.2)(p3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(0.2)(p4)

    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)
    c5 = Dropout(0.3)(c5)

    # Decoder
    u6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = Concatenate()([u6, c4])
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = Conv2D(512, (3, 3), activation='relu', padding='same')(c6)
    c6 = Dropout(0.2)(c6)

    u7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = Concatenate()([u7, c3])
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = Conv2D(256, (3, 3), activation='relu', padding='same')(c7)
    c7 = Dropout(0.2)(c7)

    u8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = Concatenate()([u8, c2])
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = Conv2D(128, (3, 3), activation='relu', padding='same')(c8)
    c8 = Dropout(0.1)(c8)

    u9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = Concatenate()([u9, c1])
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = Conv2D(64, (3, 3), activation='relu', padding='same')(c9)
    c9 = Dropout(0.1)(c9)

    outputs = Conv2D(num_classes, (1, 1), activation='sigmoid')(c9)
    model = Model(inputs, outputs)
    return model

# ================================
# 5. Compile and Train the Model
# ================================
input_shape = (img_size, img_size, 3)
model = unet_model(input_shape, num_classes=1)

model.compile(optimizer=Adam(learning_rate=ExponentialDecay(0.001, decay_steps=10000, decay_rate=0.9)),
              loss=dice_loss,
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)
model_checkpoint = ModelCheckpoint('unet_cinnamon_segmentation.h5', save_best_only=True)

# Train the model
history = model.fit(data_gen.flow(X_train, y_train, batch_size=16),
                    validation_data=(X_val, y_val),
                    epochs=50,
                    callbacks=[early_stopping, reduce_lr, model_checkpoint])

# ================================
# 6. Save the Model
# ================================
model.save('unet_cinnamon_segmentation.h5')
print("Model saved as 'unet_cinnamon_segmentation.h5'")


Loaded 369 images and 369 masks


c:\CINNAMON\Cinnamon App\Cinnamon App\python_backend\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 55s/step - accuracy: 0.0014 - loss: 0.7892 

19/19 ━━━━━━━━━━━━━━━━━━━━ 1216s 63s/step - accuracy: 0.0014 - loss: 0.7889 - val_accuracy: 0.0000e+00 - val_loss: 0.7885 - learning_rate: 9.9980e-04
Epoch 2/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1047s 54s/step - accuracy: 0.0000e+00 - loss: 0.7783 - val_accuracy: 0.0000e+00 - val_loss: 0.7885 - learning_rate: 9.9960e-04
Epoch 3/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1041s 54s/step - accuracy: 0.0000e+00 - loss: 0.7760 - val_accuracy: 0.0000e+00 - val_loss: 0.7885 - learning_rate: 9.9940e-04
Epoch 4/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1034s 54s/step - accuracy: 0.0000e+00 - loss: 0.7808 - val_accuracy: 0.0000e+00 - val_loss: 0.7885 - learning_rate: 9.9920e-04
Epoch 5/50
19/19 ━━━━━━━━━━━━━━━━━━━━ 1015s 53s/step - accuracy: 0.0000e+00 - loss: 0.7836 - val_accuracy: 0.0000e+00 - val_loss: 0.7885 - learning_rate: 9.9900e-04
Epoch 6/50
 7/19 ━━━━━━━━━━━━━━━━━━━━ 10:02 50s/step - accuracy: 0.0000e+00 - loss: 0.7765

AbortedError: Graph execution error:

Detected at node StatefulPartitionedCall/functional_1/conv2d_9_1/Relu defined at (most recent call last):
<stack traces unavailable>
Operation received an exception:Status: 1, message: could not create a memory object, in file tensorflow/core/kernels/mkl/mkl_conv_ops.cc:1112
	 [[{{node StatefulPartitionedCall/functional_1/conv2d_9_1/Relu}}]] [Op:__inference_multi_step_on_iterator_8720]

In [3]:
y_pred = model.predict(X_val)
# Ensure that y_val is binary
 #Threshold predictions to get binary masks
y_pred_thresh = (y_pred > 0.5).astype(np.uint8)
y_val_thresh = (y_val > 0.5).astype(np.uint8)

# Flatten the arrays for metric calculation
y_true_flat = y_val_thresh.flatten()
y_pred_flat = y_pred_thresh.flatten()

# Compute metrics
precision = precision_score(y_true_flat, y_pred_flat)
recall = recall_score(y_true_flat, y_pred_flat)
f1 = f1_score(y_true_flat, y_pred_flat)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# Dice Coefficient
def dice_coefficient(y_true, y_pred, smooth=1):
    intersection = np.sum(y_true * y_pred)
    return (2. * intersection + smooth) / (np.sum(y_true) + np.sum(y_pred) + smooth)

dice = dice_coefficient(y_val_thresh, y_pred_thresh)
print(f"Dice Coefficient: {dice:.4f}")

# Intersection over Union (IoU)
def iou(y_true, y_pred):
    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred) - intersection
    return intersection / union

iou_score = iou(y_val_thresh, y_pred_thresh)
print(f"IoU: {iou_score:.4f}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 62s 20s/step


NameError: name 'precision_score' is not defined

In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

# Function to preprocess image
def preprocess_image(image_path, img_size):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (img_size, img_size)) / 255.0
    image = np.expand_dims(image, axis=0)
    return image

# Load the trained model
model = load_model('unet_cinnamon_segmentation.h5')

# Path to the new image
new_image_path = r'C:\CINNAMON\test_images\test_image2.jpg'

# Preprocess the new image
img_size = 256  # Same size used during training
new_image = preprocess_image(new_image_path, img_size)

# Predict the mask
predicted_mask = model.predict(new_image)

# Threshold the predicted mask
predicted_mask = (predicted_mask > 0.5).astype(np.uint8)

# Remove the batch dimension
predicted_mask = np.squeeze(predicted_mask)

# Plot the original image and the predicted mask
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Original Image
original_image = cv2.imread(new_image_path)
original_image = cv2.resize(original_image, (img_size, img_size))
ax[0].imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
ax[0].set_title('Original Image')
ax[0].axis('off')

# Predicted Mask
ax[1].imshow(predicted_mask, cmap='gray')
ax[1].set_title('Predicted Mask')
ax[1].axis('off')

plt.show()


In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import sys
!{sys.executable} -m pip install matplotlib


# Function to preprocess image
def preprocess_image(image_path, img_size):
    image = cv2.imread(image_path)
    image = cv2.resize(image, (img_size, img_size)) / 255.0
    image = np.expand_dims(image, axis=0)
    return image

# Load the trained model
model = load_model('unet_cinnamon_segmentation.h5')

# Path to the folder containing new images
image_folder_path = r'C:\CINNAMON\test_images\Test_images'
img_size = 256  # Same size used during training

# Process each image in the folder
for image_file in os.listdir(image_folder_path):
    image_path = os.path.join(image_folder_path, image_file)

    # Preprocess the image
    new_image = preprocess_image(image_path, img_size)

    # Predict the mask
    predicted_mask = model.predict(new_image)

    # Threshold the predicted mask
    predicted_mask = (predicted_mask > 0.5).astype(np.uint8)

    # Remove the batch dimension
    predicted_mask = np.squeeze(predicted_mask)

    # Plot the original image and the predicted mask
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))

    # Original Image
    original_image = cv2.imread(image_path)
    original_image = cv2.resize(original_image, (img_size, img_size))
    ax[0].imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
    ax[0].set_title('Original Image')
    ax[0].axis('off')

    # Predicted Mask
    ax[1].imshow(predicted_mask, cmap='gray')
    ax[1].set_title('Predicted Mask')
    ax[1].axis('off')

    plt.show()
